In [1]:
import numpy as np
import pandas as pd
import tenseal as ts
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder
from concurrent.futures import ThreadPoolExecutor
import time


In [2]:
# Loading and preprocessing the data
df = pd.read_csv('payment_fraud.csv')

label_encoder = LabelEncoder()
df['paymentMethod_encoded'] = label_encoder.fit_transform(df['paymentMethod'])

features = ['accountAgeDays', 'numItems', 'localTime', 'paymentMethod_encoded', 'paymentMethodAgeDays']
X = df[features].values
y = df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
# Encryption setup with TenSEAL
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=16384, coeff_mod_bit_sizes=[60, 40, 40, 60, 60])
context.generate_galois_keys()
context.global_scale = 2 ** 40

# Encrypt training and testing data
X_train_encrypted = [ts.ckks_vector(context, x) for x in X_train]
X_test_encrypted = [ts.ckks_vector(context, x) for x in X_test]


In [4]:
# Convert encrypted data to numpy array for processing
X_train_encrypted_np = np.array(X_train_encrypted).reshape(len(X_train), -1)
X_test_encrypted_np = np.array(X_test_encrypted).reshape(len(X_test), -1)


In [5]:
# Batch prediction with optimized batch size for memory efficiency
def batch_predict(X, weights, batch_size=35):
    predictions = []
    for i in range(0, len(X), batch_size):
        batch = X[i:i + batch_size]
        batch_predictions = np.dot(batch, weights)
        predictions.extend(batch_predictions)
    return np.array(predictions).reshape(-1, 1)

# Parallel predictions with optimized threading
def parallel_predict(X, weights, num_workers=4):
    def predict_batch(batch):
        return np.dot(batch, weights)
    
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(predict_batch, X[i::num_workers]) for i in range(num_workers)]
        results = [future.result() for future in futures]
    return np.concatenate(results).reshape(-1, 1)

# Compute gradients function
def compute_gradients(X, y, predictions):
    errors = predictions - y.reshape(-1, 1)
    gradients = np.dot(X.T, errors) / len(X)
    return gradients


In [ ]:
# Training loop
batch_size = 30 #5
input_dim = X_train_encrypted_np.shape[1]  # Number of features
output_dim = 1  # Assuming binary classification
weights = np.random.randn(input_dim, output_dim)

# Define learning rate and epochs
learning_rate = 0.001
epochs = 5

for epoch in range(epochs):
    start_time = time.time()
    predictions = batch_predict(X_train_encrypted_np, weights, batch_size)
    gradients = compute_gradients(X_train_encrypted_np, y_train, predictions)
    weights -= learning_rate * gradients
    
    print(f"Epoch {epoch + 1}/{epochs} completed in {time.time() - start_time:.2f} seconds")

# Testing the model
y_pred = parallel_predict(X_test_encrypted_np, weights)
y_pred_labels = [1 if pred >= 0.5 else 0 for pred in y_pred]

# Model evaluation
accuracy = accuracy_score(y_test, y_pred_labels)
precision = precision_score(y_test, y_pred_labels)
recall = recall_score(y_test, y_pred_labels)
f1 = f1_score(y_test, y_pred_labels)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


In [3]:
def batch_predict(X, weights, batch_size=100):
    predictions = []
    for i in range(0, len(X), batch_size):
        batch = X[i:i + batch_size]
        batch_predictions = np.dot(batch, weights)
        predictions.extend(batch_predictions)
    return np.array(predictions)

def parallel_predict(X, weights, num_workers=4):
    def predict_batch(batch):
        return np.dot(batch, weights)
    
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(predict_batch, X[i::num_workers]) for i in range(num_workers)]
        results = [future.result() for future in futures]
    return np.concatenate(results)

def compute_gradients(X, y, predictions):
    errors = predictions - y.reshape(-1, 1)
    gradients = np.dot(X.T, errors) / len(X)
    return gradients

In [5]:
# Initialize weights (example for a simple linear model)
input_dim = X_train.shape[1]
output_dim = 1

weights = np.random.randn(input_dim, output_dim)
learning_rate = 0.5
epochs = 2000

for epoch in range(epochs):
    predictions = batch_predict(X_train, weights)
    gradients = compute_gradients(X_train, y_train, predictions)
    weights = weights - learning_rate * gradients
    print(f"Epoch {epoch+1}/{epochs}, Weights: {weights}")

# Check predictions
y_pred = parallel_predict(X_test, weights)
print(f"Predictions: {y_pred}")

y_pred_labels = [1 if pred >= 0.5 else 0 for pred in y_pred]
print(f"Predicted Labels: {y_pred_labels}")

accuracy = accuracy_score(y_test, y_pred_labels)
precision = precision_score(y_test, y_pred_labels)
recall = recall_score(y_test, y_pred_labels)
f1 = f1_score(y_test, y_pred_labels)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Epoch 1/2000, Weights: [[4.40623975e+05]
 [2.93455823e+02]
 [1.30807962e+03]
 [9.15294389e+01]
 [6.77831051e+04]]
Epoch 2/2000, Weights: [[-3.11069252e+11]
 [-2.06033337e+08]
 [-9.16319106e+08]
 [-6.39550240e+07]
 [-4.30617319e+10]]
Epoch 3/2000, Weights: [[2.19173832e+17]
 [1.45124047e+14]
 [6.45512480e+14]
 [4.50526563e+13]
 [3.01718624e+16]]
Epoch 4/2000, Weights: [[-1.54410717e+23]
 [-1.02240185e+20]
 [-4.54767783e+20]
 [-3.17398479e+19]
 [-2.12505070e+22]]
Epoch 5/2000, Weights: [[1.08783751e+29]
 [7.20290839e+25]
 [3.20387894e+26]
 [2.23609998e+25]
 [1.49709652e+28]]
Epoch 6/2000, Weights: [[-7.66391208e+34]
 [-5.07451288e+31]
 [-2.25716118e+32]
 [-1.57535230e+31]
 [-1.05471708e+34]]
Epoch 7/2000, Weights: [[5.39929420e+40]
 [3.57503943e+37]
 [1.59019012e+38]
 [1.10984970e+37]
 [7.43057534e+39]]
Epoch 8/2000, Weights: [[-3.80385077e+46]
 [-2.51864706e+43]
 [-1.12030308e+44]
 [-7.81898979e+42]
 [-5.23490638e+45]]
Epoch 9/2000, Weights: [[2.67984668e+52]
 [1.77440925e+49]
 [7.89263

C:\Users\matth\AppData\Local\Temp\ipykernel_19772\893645143.py:12: RuntimeWarning: invalid value encountered in subtract
  weights = weights - learning_rate * gradients


Epoch 70/2000, Weights: [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Epoch 71/2000, Weights: [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Epoch 72/2000, Weights: [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Epoch 73/2000, Weights: [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Epoch 74/2000, Weights: [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Epoch 75/2000, Weights: [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Epoch 76/2000, Weights: [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Epoch 77/2000, Weights: [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Epoch 78/2000, Weights: [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Epoch 79/2000, Weights: [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Epoch 80/2000, Weights: [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Epoch 81/2000, Weights: [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Epoch 82/2000, Weights: [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Epoch 83/2000, Weights: [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Epoch 84/2000, Weights: [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Epoch 85/2000, Weights: [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Epoch 86/2000, Weights: [[nan]
 [nan]
 [

c:\ProgramData\miniconda3\envs\CVvid\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
